<a href="https://colab.research.google.com/github/Won20/Big-Data/blob/main/LR2_Dubman__Reports%20with%20Apache%20Spark/L2_Dubman_Reports_with_Apache_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Устанавливаем pyspark

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=975d2e13f3cc12f252d50e9ca120ba0cefbb5f41dc87865535b471ffda683ad2
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


# 2) Импортируем необходимые библиотеки и создаем проект

In [2]:
import pyspark
import os

from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType, StructField, StringType, DateType

In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell' #представляет переменные среды окружения системы Этот пакет можно добавить в Spark с помощью параметра командной строки --packages,
# чтобы включить его при запуске оболочки Spark. Этот пакет позволяет читать XML-файлы в локальной или распределенной файловой системе как «Spark DataFrames».
sc = SparkSession.builder.appName("L2").master("local[*]").getOrCreate() #Точкой входа во все функции Spark является класс SparkSession. Чтобы создать базовый SparkSession, используют SparkSession.builder.

In [4]:
sc

# 3) Загружаем файл с именами языков программирования

In [5]:
languages = sc.read.csv("/content/data/programming-languages.csv")
#создание DataFrame на основе содержимого файла

In [6]:
# Создадим список из имен языков программирования
languages_list = [str(x[0]) for x in languages.collect()]
#collect -  используется для извлечения всех элементов со всех узлов в узел драйвера

In [7]:
languages_list.pop(0)# удаляет из списка последний элемент и возвращает его значениe

'name'

In [8]:
languages_list[:10] #выводим первые 10  значений 'name' из файла языков

['A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET',
 'ABSYS']

# 4) Загружаем файл с выборкой данных по использованию языков программирования

In [9]:
posts_sample = sc.read.format("xml").options(rowTag="row").load('/content/data/posts_sample.xml')#После запуска оболочки Pyspark вместе с указанным пакетом XML получаем доступ к файлу данных XML с помощью API Dataframe

In [10]:
posts_sample.take(5)
#возвращает первые 5 элементов в виде списка строк

[Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUs

## 5) Функционал программы необходимый для поиска

### 1) Функция для фильтрации по датам

In [11]:
def check_date(x, year):
  """
  Данная функция была написана для фильтрации по датам,
  так как нас интересует период с 2010 год по 2020 (не включая) год
  """
  start = datetime(year=year, month=1, day=1)
  end = datetime(year=year, month=12, day=31)
  CreationDate = x._CreationDate
  return CreationDate >= start and CreationDate <= end

### 2) Функция перевода текста в нижний регистр и поиска названия языка программирования

In [12]:

def language_detection(x):
  """
  Данная функция переводит весь текст в нижний регистр
  и ищет название языка программирования в каждой строке,
  если язык был найден, то создается кортеж, иначе None
  """
  tag = None
  for language in languages_list:
    if "<" + language.lower() + ">" in x._Tags.lower():
      tag = language
      break
  if tag is None:
    return None
  return (x._Id, tag)



### 3) Код для непосредственного поиска

In [13]:
"""
Данный кусок кода сначала убирает пустые значения и оставляет диапазон с 2010 по 2020 (не включая) год,
далее мы находим язык программирования в каждой строке и убираем пустые значения, если не был
найден, потом смотрим, сколько раз упоминался каждый язык программирования в каждом годе и сортируем по
количеству повторений и в конце идет сортировка от большего к меньшему по количеству упоминаний.
"""

final_result = {}
for year in range(2010, 2020):
  final_result[year] = posts_sample.rdd\
      .filter(lambda x: x._Tags is not None and check_date(x, year))\
      .map(language_detection)\
      .filter(lambda x: x is not None)\
      .keyBy(lambda x: x[1])\
      .aggregateByKey(
          0,
          lambda x, y: x + 1,
          lambda x1, x2: x1 + x2,
      )\
      .sortBy(lambda x: x[1], ascending=False)\
      .toDF()
  final_result[year] = final_result[year].select(col("_1").alias("Programming_language"),
                                                 col("_2").alias(f"Number_of_mentions_in_{year}")).limit(10)
  final_result[year].show()
  #rdd - устойчивый распределенный набор данных, базовая абстракция в Spark. Представляет неизменяемую секционированную
  ##коллекцию элементов, с которыми можно работать параллельно. Возвращает содержимое в виде pyspark.RDD строки.
  #filter - возвращает новый RDD, содержащий только элементы, удовлетворяющие предикату, то есть фильтрует строки, используя заданное условие.
  #map - возвращает новый RDD, применив функцию к каждому элементу этого RDD.
  #keyBy - создает кортежи элементов в этом RDD, применяя x.
  #aggregateByKey - агригирует значения каждого ключа, используя заданные функции объединения и нейтральное «нулевое значение».
  ##эта функция может возвращать тип результата U = 0, отличный от типа значений в этом RDD, V.
  ##таким образом, нам нужна одна операция для слияния V с U и одна операция для слияния двух U.
  ##первая операция используется для объединение значений внутри раздела, а последний используется для объединения значений между разделами.
  ##чтобы избежать выделения памяти, обеим этим функциям разрешено изменять и возвращать свой первый аргумент вместо создания нового U.
  #sortBy - сортирует этот RDD по заданной ключевой функции.
  #toDF - возвращает новый DataFrame
  #select - проецирует набор выражений и возвращает новый DataFrame.

+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2010|
+--------------------+--------------------------+
|                Java|                        52|
|          JavaScript|                        44|
|                 PHP|                        42|
|              Python|                        25|
|         Objective-C|                        22|
|                   C|                        20|
|                Ruby|                        11|
|              Delphi|                         7|
|                   R|                         3|
|                Bash|                         3|
+--------------------+--------------------------+

+--------------------+--------------------------+
|Programming_language|Number_of_mentions_in_2011|
+--------------------+--------------------------+
|                 PHP|                        97|
|                Java|                        92|
|          JavaScript|                        82|

### 6) Запись результата в файл

In [14]:
#запись результатов в файл
for year in final_result.keys():
    final_result[year].write.format("parquet").save(f"/content/final_result/top_{year}")